In [1]:
from library.data import *
from library.synonyms import *
import nltk

In [2]:
ld = LoadData()

Loading topics

In [4]:
topicn,query = ld.load_topics()

Loading Corpus

In [5]:
corpus = ld.load_corpus()


Loading query results

In [6]:
qtopicn,idn,label = ld.load_query_results()

for each query, we pass to the ir engine, which scans the corpus and returns query results. We then compare these query results against the query results data.

because of the way the algorithm works, we need to collect synonyms up to num_synonyms 

We also need to normalize the resulting figure to take into account that some words will have many synonyms or big clusters, and some will have few. The scores for the ones with few synonyms, and long documents, will necessarily be lower, but that doesn't necessarily mean they are less relevant. Thus there ought to be a normalization factor which accounts for the number of synonyms/size of cluster and also the length of the document. (num unique tokens)

We also need to have a maximum number of synonyms/clustered selected words as a parameter.

add something to increase the number of synonyms, and add an associated discount factor for each synonym "level" you go out

Do hyperparameter tuning via a grid search. Do a train test split as well, as you are effectively "training" the model by tuning the hyperparameters, and you don't want to overfit the hyperparameters to your results and then report those.

Also do auc curve by varying decision boundary

can you determine correct decision threshold by looking at the distribution of confidence values in the population? If you know that 5% of the documents will be relevant, perhaps ste the decision threshold to include 5% of documents? Or maybe 10%? If you know the precision and recall curves, then maybe you can create a function that takes in the auc curve, the percent estimated to be positivei in the population, and gives you predicted precision, recall, and accuracy, for each given decision boundary placement? This might be a proposal for discussion/future research section.

In [6]:
syn = Synonyms()

In [7]:
syn.get('intelligence')

['acumen',
 'agility',
 'brilliance',
 'intellect',
 'judgment',
 'perception',
 'quickness',
 'savvy',
 'sense',
 'skill',
 'subtlety',
 'understanding',
 'wit',
 'IQ',
 'acuity',
 'alertness',
 'aptitude',
 'brainpower',
 'brains',
 'brightness',
 'capacity',
 'cleverness',
 'comprehension',
 'coruscation',
 'discernment',
 'luminosity',
 'mentality',
 'mind',
 'penetration',
 'perspicacity',
 'precocity',
 'quotient',
 'reason',
 'sagacity',
 'smarts',
 'trenchancy']

In [112]:
class SynonymIR:
    def __init__(self,max_similar=20,similar_significance=0.5):
        """ 
        
        NOTE: I need to add the functionality to get synonyms of 
              synonyms in order to fill out the graph in contexts
              where there are relatively few synonyms.
        """
        self.__syn = Synonyms()
        self.__ps = nltk.stem.PorterStemmer()
        self.__max_similar = max_similar
        self.__similar_significance = similar_significance

    def preprocess_document(self,document):
        document = document.replace('-',' ')
        document = ''.join([i for i in document if (i.isalpha()) or (i == ' ')])
        document = document.lower()
        document = nltk.word_tokenize(document)
        document = list(set(document))
        # remove stopwords
        document = [word for word in document if not word in nltk.corpus.stopwords.words()]
        document = [self.__ps.stem(w) for w in document]
        # stem
        return set(document)

    def preprocess_query(self,query):
        # need to remove stopwords from query
        query = [i for i in query.split() if i not in nltk.corpus.stopwords.words()]
        # need to get synonyms
        synonyms = []
        for word in query:
            synonyms.extend(self.__syn.get(word))
    
        # need to tokenize and stem synonyms and query
        query = [self.__ps.stem(i) for i in query]
        synonyms = list(set([self.__ps.stem(i) for i in synonyms]))
        if len(synonyms) > self.__max_similar:
            synonyms = synonyms[:self.__max_similar] # not a great selection mechanism, but not sure about alternative
        # need to return each as a set
        return set(query), set(synonyms)

    def get_jaccard_score(self,A,B):
        intersection = A.intersection(B)
        symmetric_difference = A ^ B
        return len(intersection)/(len(intersection)+len(symmetric_difference))
    
    def similarity_score(self,query_jaccard,synonyms_jaccard,similar_significance):
        return (query_jaccard*(1-similar_significance))+(synonyms_jaccard*similar_significance)

    def get_similarity(self,query,document):
        query,synonyms = self.preprocess_query(query)
        document = self.preprocess_document(document)
        query_jaccard = self.get_jaccard_score(query,document)
        synonyms_jaccard = self.get_jaccard_score(synonyms,document)
        # we want the similar_significance to be between 0 and 1
        # at 0, the query jaccard score should be 100% of the score
        # at 1, the synonyms jaccard score should be 100% of the score
        similarity_score = self.similarity_score(query_jaccard,synonyms_jaccard,self.__similar_significance)
        return similarity_score

In [113]:
sir = SynonymIR()

In [114]:
n = 0

In [120]:
tq = query[0]
td = corpus[0]['text']

In [121]:
tq,ts = sir.preprocess_query(tq)
td = sir.preprocess_document(td)

In [124]:
td

{'ab',
 'across',
 'adher',
 'among',
 'andor',
 'anticip',
 'antigen',
 'arm',
 'attack',
 'background',
 'base',
 'case',
 'chang',
 'circul',
 'clinic',
 'clinicaltrialsgov',
 'cluster',
 'collect',
 'complet',
 'compos',
 'conclusionssignific',
 'conduct',
 'confirm',
 'contact',
 'control',
 'correspond',
 'countri',
 'cultur',
 'data',
 'day',
 'definit',
 'depend',
 'design',
 'diagnos',
 'differ',
 'drift',
 'due',
 'durat',
 'efficaci',
 'feasibl',
 'find',
 'follow',
 'hand',
 'hkclinicaltrialscom',
 'hkctr',
 'home',
 'hong',
 'hour',
 'household',
 'hygien',
 'ill',
 'implement',
 'index',
 'influenza',
 'inform',
 'intervent',
 'issu',
 'kong',
 'laboratori',
 'lack',
 'later',
 'learnt',
 'least',
 'lesson',
 'like',
 'lower',
 'main',
 'mask',
 'measur',
 'member',
 'methodologyprincip',
 'nct',
 'nose',
 'outcom',
 'pattern',
 'perhap',
 'pharmaceut',
 'pilot',
 'present',
 'primari',
 'quickvu',
 'random',
 'rapid',
 'ratio',
 'recent',
 'recruit',
 'reduc',
 'registr'

In [110]:
sir.get_similarity(query[n],corpus[11]['text'])

0.0

In [111]:
print(query[n],'\n\n',corpus[11]['text'])

coronavirus origin 

 BACKGROUND: Pediatric LRTI hospitalizations are a significant burden on patients, families, and healthcare systems. This study determined the burden of pediatric LRTIs on hospital settings in British Columbia and the benefits of prevention strategies as they relate to healthcare resource demand. METHODS: LRTI inpatient episodes for patients <19 years of age during 2008–2010 were extracted from the BC Discharge Abstract Database. The annual number of acute care beds required to treat pediatric LRTIs was estimated. Sub-analyses determined the burden due to infants <1 year of age and high-risk infants. Population projections were used to forecast LRTI hospitalizations and the effectiveness of public health initiatives to reduce the incidence of LRTIs to 2020 and 2030. RESULTS: During 2008–2010, LRTI as the primary diagnosis accounted for 32.0 and 75.9% hospitalizations for diseases of the respiratory system in children <19 years of age and infants <1 year of age, res

In [65]:
qj = 0.1
sj = 0.4
ss = 0.5

In [66]:
(qj*(1-ss))+(sj*ss)

0.25

In [59]:
sj*(ss)

0.4

In [17]:
(qj+sj)/(1/ss)

0.6000000000000001

In [8]:
a = {1,2,3}
b = {2,3,4}

In [11]:
len(b.intersection(a))

2

In [14]:
ex = corpus[0]['text']

Get set intersection. Discount factor determines the divisor for the jaccard score of the either synonyms, or clustered words

In [37]:
ex = preprocess(ex)

{'chang', 'coronaviru', 'respons', 'weather'}